Q1. Gender Classification

Use train_test_split(random_state = 42,test_size = 0.2)

Do some EDA (countplots, show images, find missing values etc.)

a. Use Base SVM(LinearSVC) model.

b. Create a CNN model with 3 Conv2D layers, 3 Maxpooling2D layers, any extra layers necessary.

c. Get atleast 90% test accuracy using the above CNN model.

df: https://www.kaggle.com/dfsets/cashutosh/gender-classification-dfset

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
import cv2


In [ ]:
male = "./Training/male/"
female = "./Training/female/"

In [ ]:
male_files = [os.path.join(male, file) for file in os.listdir(male)]
female_files = [os.path.join(female, file) for file in os.listdir(female)]

# into dataframe
df = pd.DataFrame({"image_path": male_files + female_files, "gender": ["male"] * len(male_files) + ["female"] * len(female_files)})

In [ ]:
# print(df.info())

display(df)

,image_path,gender
0,./Training/male/090544.jpg.jpg,male
1,./Training/male/090545.jpg.jpg,male
2,./Training/male/090548.jpg.jpg,male
3,./Training/male/090550.jpg.jpg,male
4,./Training/male/090553.jpg.jpg,male
...,...,...
47004,./Training/female/202592.jpg.jpg,female
47005,./Training/female/202593.jpg.jpg,female
47006,./Training/female/202594.jpg.jpg,female
47007,./Training/female/202598.jpg.jpg,female


In [ ]:
# seting the dataset
X = df['image_path']
y = df['gender']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Load images and convert them to grayscale
image_size = (48, 48)
X_train_images = X_train.apply(lambda x: cv2.resize(cv2.cvtColor(cv2.imread(x), cv2.COLOR_BGR2GRAY), image_size))
X_test_images = X_test.apply(lambda x: cv2.resize(cv2.cvtColor(cv2.imread(x), cv2.COLOR_BGR2GRAY), image_size))

# Flatten images
X_train_flat = np.array([img.flatten() for img in X_train_images])
X_test_flat = np.array([img.flatten() for img in X_test_images])

# Initialize and train LinearSVC
svm_model = LinearSVC(random_state=42)
svm_model.fit(X_train_flat, y_train)

# Predict and evaluate
svm_pred = svm_model.predict(X_test_flat)
svm_accuracy = accuracy_score(y_test, svm_pred)
print("LinearSVC Accuracy:", svm_accuracy)

c:\Users\hp2023\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\hp2023\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC Accuracy: 0.8114231014677729


In [ ]:
# Convert gender labels to numerical format
gender_mapping = {"male": 0, "female": 1}
y_train_numeric = y_train.map(gender_mapping)
y_test_numeric = y_test.map(gender_mapping)

# Load images and preprocess them for CNN
X_train_cnn = X_train.apply(lambda x: np.array(Image.open(x).resize((48, 48))).astype('float32') / 255.0)
X_test_cnn = X_test.apply(lambda x: np.array(Image.open(x).resize((48, 48))).astype('float32') / 255.0)

# Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(np.array(X_train_cnn.tolist()), y_train_numeric, epochs=10, batch_size=32, validation_split=0.1)

# Evaluate the model
cnn_loss, cnn_accuracy = model.evaluate(np.array(X_test_cnn.tolist()), y_test_numeric)
print("CNN Test Accuracy:", cnn_accuracy)



Epoch 1/10
1058/1058 [==============================] - 141s 126ms/step - loss: 0.2549 - accuracy: 0.8942 - val_loss: 0.1714 - val_accuracy: 0.9378
Epoch 2/10
1058/1058 [==============================] - 126s 119ms/step - loss: 0.1556 - accuracy: 0.9435 - val_loss: 0.1599 - val_accuracy: 0.9476
Epoch 3/10
1058/1058 [==============================] - 130s 123ms/step - loss: 0.1338 - accuracy: 0.9501 - val_loss: 0.1448 - val_accuracy: 0.9431
Epoch 4/10
1058/1058 [==============================] - 127s 120ms/step - loss: 0.1174 - accuracy: 0.9573 - val_loss: 0.1335 - val_accuracy: 0.9489
Epoch 5/10
1058/1058 [==============================] - 133s 125ms/step - loss: 0.1104 - accuracy: 0.9588 - val_loss: 0.1183 - val_accuracy: 0.9614
Epoch 6/10
1058/1058 [==============================] - 130s 122ms/step - loss: 0.0985 - accuracy: 0.9648 - val_loss: 0.1195 - val_accuracy: 0.9599
Epoch 7/10
1058/1058 [==============================] - 129s 122ms/step - loss: 0.0933 - accuracy: 0.9666 - val_